### Imports

In [2]:
import gc
import re
from typing import List, Dict, Tuple

import numpy as np
import polars as pl
import polars.selectors as cs

### Training Data

In [87]:
# data = pl.concat([train, pl.read_parquet('/kaggle/input/aeroclub-recsys-2025/test.parquet')])
data = pl.read_parquet('/kaggle/input/aeroclub-recsys-2025/train.parquet')

In [4]:
data.head(1000)

Id,bySelf,companyID,corporateTariffCode,frequentFlyer,nationality,isAccess3D,isVip,legs0_arrivalAt,legs0_departureAt,legs0_duration,legs0_segments0_aircraft_code,legs0_segments0_arrivalTo_airport_city_iata,legs0_segments0_arrivalTo_airport_iata,legs0_segments0_baggageAllowance_quantity,legs0_segments0_baggageAllowance_weightMeasurementType,legs0_segments0_cabinClass,legs0_segments0_departureFrom_airport_iata,legs0_segments0_duration,legs0_segments0_flightNumber,legs0_segments0_marketingCarrier_code,legs0_segments0_operatingCarrier_code,legs0_segments0_seatsAvailable,legs0_segments1_aircraft_code,legs0_segments1_arrivalTo_airport_city_iata,legs0_segments1_arrivalTo_airport_iata,legs0_segments1_baggageAllowance_quantity,legs0_segments1_baggageAllowance_weightMeasurementType,legs0_segments1_cabinClass,legs0_segments1_departureFrom_airport_iata,legs0_segments1_duration,legs0_segments1_flightNumber,legs0_segments1_marketingCarrier_code,legs0_segments1_operatingCarrier_code,legs0_segments1_seatsAvailable,legs0_segments2_aircraft_code,legs0_segments2_arrivalTo_airport_city_iata,…,legs1_segments2_baggageAllowance_weightMeasurementType,legs1_segments2_cabinClass,legs1_segments2_departureFrom_airport_iata,legs1_segments2_duration,legs1_segments2_flightNumber,legs1_segments2_marketingCarrier_code,legs1_segments2_operatingCarrier_code,legs1_segments2_seatsAvailable,legs1_segments3_aircraft_code,legs1_segments3_arrivalTo_airport_city_iata,legs1_segments3_arrivalTo_airport_iata,legs1_segments3_baggageAllowance_quantity,legs1_segments3_baggageAllowance_weightMeasurementType,legs1_segments3_cabinClass,legs1_segments3_departureFrom_airport_iata,legs1_segments3_duration,legs1_segments3_flightNumber,legs1_segments3_marketingCarrier_code,legs1_segments3_operatingCarrier_code,legs1_segments3_seatsAvailable,miniRules0_monetaryAmount,miniRules0_percentage,miniRules0_statusInfos,miniRules1_monetaryAmount,miniRules1_percentage,miniRules1_statusInfos,pricingInfo_isAccessTP,pricingInfo_passengerCount,profileId,ranker_id,requestDate,searchRoute,sex,taxes,totalPrice,selected,__index_level_0__
i64,bool,i64,i64,str,i64,bool,bool,str,str,str,str,str,str,f64,f64,f64,str,str,str,str,str,f64,str,str,str,f64,f64,f64,str,str,str,str,str,f64,str,str,…,f64,f64,str,str,str,str,str,f64,str,str,str,f64,f64,f64,str,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,str,datetime[ns],str,bool,f64,f64,i64,i64
0,true,57323,null,"""S7/SU/UT""",36,false,false,"""2024-06-15T16:20:00""","""2024-06-15T15:40:00""","""02:40:00""","""YK2""","""KJA""","""KJA""",1.0,0.0,1.0,"""TLK""","""02:40:00""","""216""","""KV""","""KV""",9.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1.0,1,2087645,"""98ce0dabf6964640b63079fbafd42c…",2024-05-17 03:03:08,"""TLKKJA/KJATLK""",true,370.0,16884.0,1,0
1,true,57323,123,"""S7/SU/UT""",36,true,false,"""2024-06-15T14:50:00""","""2024-06-15T09:25:00""","""07:25:00""","""E70""","""OVB""","""OVB""",1.0,0.0,1.0,"""TLK""","""02:50:00""","""5358""","""S7""","""S7""",4.0,"""E70""","""KJA""","""KJA""",1.0,0.0,1.0,"""OVB""","""01:20:00""","""5311""","""S7""","""S7""",4.0,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2300.0,null,1.0,3500.0,null,1.0,1.0,1,2087645,"""98ce0dabf6964640b63079fbafd42c…",2024-05-17 03:03:08,"""TLKKJA/KJATLK""",true,2240.0,51125.0,0,1
2,true,57323,null,"""S7/SU/UT""",36,false,false,"""2024-06-15T14:50:00""","""2024-06-15T09:25:00""","""07:25:00""","""E70""","""OVB""","""OVB""",1.0,0.0,1.0,"""TLK""","""02:50:00""","""5358""","""S7""","""S7""",4.0,"""E70""","""KJA""","""KJA""",1.0,0.0,1.0,"""OVB""","""01:20:00""","""5311""","""S7""","""S7""",4.0,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2300.0,null,1.0,3500.0,null,1.0,1.0,1,2087645,"""98ce0dabf696

In [32]:
code_counts = (data
    .lazy()
    .select(pl.col('frequentFlyer'))
    .filter(pl.col('frequentFlyer').is_not_null())
    .with_columns([pl.col('frequentFlyer').str.replace('- ЮТэйр ЗАО', 'UT')])
    .select(pl.col('frequentFlyer').str.split('/').list.explode().alias('code'))
    .group_by('code')
    .agg(pl.len().alias('count'))
    .with_columns([(pl.col('count') / pl.col('count').sum() * 100).alias('percentage')])
    .sort('count', descending=True)
    .collect()
)
code_counts

code,count,percentage
str,u32,f64
"""SU""",5879123,63.985022
"""S7""",1839799,20.023323
"""UT""",396461,4.314855
"""U6""",266287,2.898116
"""TK""",257099,2.798119
…,…,…
"""9X""",43,0.000468
"""A3""",30,0.000327
"""WY""",14,0.000152


### Utilities

In [5]:
MAJOR_HUBS = ['ATL','DXB','DFW','HND','LHR','DEN','ORD','IST','PVG','ICN','CDG', 'JFK','CLT','MEX','SFO','EWR','MIA','BKK','GRU','HKG']

def camel_to_snake(name):
    """Convert camelCase or PascalCase to snake_case"""
    s1 = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
    s2 = re.sub('([a-z0-9])([A-Z])', r'\1_\2', s1)
    return s2.lower()


def convert_columns_to_snake_case(df):
    """Convert all column names in a polars DataFrame to snake_case"""
    return df.rename({col: camel_to_snake(col) for col in df.columns})


def parse_duration_to_minutes(duration_col: str) -> pl.Expr:
    """Parse duration string to minutes (handles format like '2.04:20' - D.HH:MM)."""
    return (
        pl.when(pl.col(duration_col).is_not_null() & (pl.col(duration_col) != ""))
        .then(
            pl.when(pl.col(duration_col).str.contains(r'\.'))
            .then(
                # Format: D.HH:MM:SS (e.g., "1.00:30:00", "2.09:45:00")
                pl.col(duration_col).str.extract(r'^(\d+)\.(\d{2}):(\d{2}):(\d{2})$', 1).cast(pl.Int32, strict=False).fill_null(0) * 1440 +  # Days
                pl.col(duration_col).str.extract(r'^(\d+)\.(\d{2}):(\d{2}):(\d{2})$', 2).cast(pl.Int32, strict=False).fill_null(0) * 60 +   # Hours
                pl.col(duration_col).str.extract(r'^(\d+)\.(\d{2}):(\d{2}):(\d{2})$', 3).cast(pl.Int32, strict=False).fill_null(0) +        # Minutes
                (pl.col(duration_col).str.extract(r'^(\d+)\.(\d{2}):(\d{2}):(\d{2})$', 4).cast(pl.Int32, strict=False).fill_null(0) / 60).round(0).cast(pl.Int32, strict=False)  # Seconds
            )
            .otherwise(
                # Format: HH:MM:SS (e.g., "07:25:00", "17:55:00")
                pl.col(duration_col).str.extract(r'^(\d{2}):(\d{2}):(\d{2})$', 1).cast(pl.Int32, strict=False).fill_null(0) * 60 +   # Hours
                pl.col(duration_col).str.extract(r'^(\d{2}):(\d{2}):(\d{2})$', 2).cast(pl.Int32, strict=False).fill_null(0) +        # Minutes
                (pl.col(duration_col).str.extract(r'^(\d{2}):(\d{2}):(\d{2})$', 3).cast(pl.Int32, strict=False).fill_null(0) / 60).round(0).cast(pl.Int32, strict=False)  # Seconds
            )
        )
        .otherwise(0)
    )

### Engineer Customer Features

In [6]:
# Customer attributes for clustering analysis

# def get_cabin_class_columns(df: pl.DataFrame) -> List[str]:
#     """Get all cabin class columns from the dataframe."""
#     columns = df.columns
#     return [col for col in columns if col.startswith('legs') and col.endswith('_cabinClass')]


def create_customer_aggregation_features() -> List[pl.Expr]:
    """Create customer aggregation expressions for basic attributes and search behavior."""
    return [
        # Basic customer attributes (take first non-null value per customer)
        pl.col('companyID').drop_nulls().first().alias('company_id'),
        pl.col('sex').drop_nulls().first().cast(pl.Int8).alias('sex'),
        pl.col('nationality').drop_nulls().first().alias('nationality'),
        pl.col('frequentFlyer').drop_nulls().first().str.replace('- ЮТэйр ЗАО', 'UT').fill_null('').alias('frequent_flyer'),
        pl.col('isVip').drop_nulls().first().cast(pl.Int8).alias('is_vip'),
        pl.col('bySelf').drop_nulls().first().cast(pl.Int8).alias('by_self'),
        pl.col('corporateTariffCode').is_not_null().cast(pl.Int8).max().alias('has_corp_codes'),

        # Search behavior metrics
        pl.len().alias('total_searches'),
        pl.col('legs1_departureAt').is_not_null().mean().alias('roundtrip_preference'),
        pl.col('searchRoute').drop_nulls().n_unique().alias('unique_routes_searched'),
    ]


def create_booking_lead_time_features() -> List[pl.Expr]:
    """Create booking lead time statistics."""
    # Calculate booking lead time in days
    booking_lead_expr = (
        (pl.col('legs0_departureAt').str.to_datetime() -
         pl.col('requestDate').cast(pl.Datetime)) / pl.duration(days=1)
    ).cast(pl.Int32)

    return [
        booking_lead_expr.min().alias('min_booking_lead_days'),
        booking_lead_expr.max().alias('max_booking_lead_days'),
        booking_lead_expr.mean().alias('avg_booking_lead_days'),
        booking_lead_expr.median().alias('median_booking_lead_days'),
    ]


def create_travel_preference_features() -> List[pl.Expr]:
    """Create travel preference features for most common airports and carriers."""
    return [
        # Most common departure airport
        pl.col('legs0_segments0_departureFrom_airport_iata').drop_nulls().mode().first().alias('most_common_departure_airport'),
        pl.col('legs0_segments0_departureFrom_airport_iata').drop_nulls().n_unique().alias('unique_departure_airports'),

        # Most common marketing carrier
        pl.col('legs0_segments0_marketingCarrier_code').drop_nulls().mode().first().alias('most_common_carrier'),
        pl.col('legs0_segments0_marketingCarrier_code').drop_nulls().n_unique().alias('unique_carriers_used'),
    ]


def create_cabin_class_features(cabin_class_cols: List[str]) -> List[pl.Expr]:
    """Create cabin class preference statistics."""
    if not cabin_class_cols:
        # Return default values if no cabin class columns found
        return [
            pl.lit(None).alias('min_cabin_class'),
            pl.lit(None).alias('max_cabin_class'),
            pl.lit(None).alias('avg_cabin_class'),
        ]

    return [
        # Cabin class statistics across all segments
        pl.min_horizontal([pl.col(col) for col in cabin_class_cols]).min().alias('min_cabin_class'),
        pl.max_horizontal([pl.col(col) for col in cabin_class_cols]).max().alias('max_cabin_class'),
        pl.mean_horizontal([pl.col(col) for col in cabin_class_cols]).mean().alias('avg_cabin_class'),
    ]


def create_temporal_preference_features() -> List[pl.Expr]:
    """Create temporal preference features for departure patterns."""
    return [
        # Weekday preference (most common day of week for departures)
        pl.col('legs0_departureAt').str.to_datetime().dt.weekday()
          .mode().first().alias('weekday_preference'),

        # Weekend travel rate (percentage of weekend departures - 5=Sat, 6=Sun)
        pl.col('legs0_departureAt').str.to_datetime().dt.weekday()
          .map_elements(lambda x: 1 if x >= 5 else 0, return_dtype=pl.Int8)
          .mean().alias('weekend_travel_rate'),

        # Time of day variance (how consistent are their departure times)
        pl.col('legs0_departureAt').str.to_datetime().dt.hour()
          .std().alias('time_of_day_variance'),

        # Night flight preference (flights departing 22:00-06:00)
        pl.col('legs0_departureAt').str.to_datetime().dt.hour()
          .map_elements(lambda x: 1 if (x >= 22 or x < 6) else 0, return_dtype=pl.Int8)
          .mean().alias('night_flight_preference')
    ]


def create_route_specific_features() -> List[pl.Expr]:
    """Create features related to route preferences and characteristics."""
    return [
        # Route loyalty
        (1 - (pl.col('searchRoute').n_unique() / pl.len().clip(1, None))).alias('route_loyalty'),

        # Hub preference (preference for major hub airports)
        (
            pl.col('legs0_segments0_departureFrom_airport_iata').is_in(MAJOR_HUBS) |
            pl.col('legs0_segments0_arrivalTo_airport_iata').is_in(MAJOR_HUBS)
        ).cast(pl.Int8).mean().alias('hub_preference'),

        # Short haul preference
        ((1 - (pl.col('leg0_duration_minutes').mean() / 180)) * 0.7 +
         pl.when(pl.col('leg0_duration_minutes') <= 180).then(1).otherwise(0).mean() * 0.3)
        .clip(0, 1).alias('short_haul_preference'),

        # Connection tolerance (average segments)
        pl.sum_horizontal([
            pl.col(f'legs0_segments{i}_departureFrom_airport_iata').is_not_null().cast(pl.Int8)
            for i in range(4)  # assuming max 4 segments
        ]).mean().alias('connection_tolerance'),

        # Preference for longer vs shorter flights (duration quartile preference)
        pl.when(pl.col('leg0_duration_minutes') <= pl.col('leg0_duration_q25'))
        .then(1)  # Short flights
        .when(pl.col('leg0_duration_minutes') <= pl.col('leg0_duration_q50'))
        .then(2)  # Medium-short flights
        .when(pl.col('leg0_duration_minutes') <= pl.col('leg0_duration_q75'))
        .then(3)  # Medium-long flights
        .otherwise(4)  # Long flights
        .mode().first().alias('preferred_duration_quartile')
    ]


def create_price_sensitivity_features() -> List[pl.Expr]:
    """Create features related to price sensitivity and patterns."""
    return [
        # Basic correlation between price and duration
        pl.corr(
            pl.col('totalPrice'),
            pl.col('trip_duration_minutes')
        ).fill_null(0).alias('price_to_duration_sensitivity'),

        # Price per minute metric (average across all flights)
        (pl.col('totalPrice') / pl.col('trip_duration_minutes').clip(1, None)).mean().alias('avg_price_per_minute'),

        # Consistency of price-per-minute (lower std = more consistent valuation)
        (pl.col('totalPrice') / pl.col('trip_duration_minutes').clip(1, None)).std().fill_null(0).alias('price_per_minute_variance'),

        # Price position within searches (percentile rank)
        pl.col('price_percentile').mean().alias('price_position_preference'),

        # Premium economy preference (assuming cabin class 2 is premium economy)
        pl.mean_horizontal([
            pl.col(f'legs0_segments{i}_cabinClass') == 2
            for i in range(4)
        ]).mean().alias('premium_economy_preference'),

        # Consistent price tier
        (1 - (
            pl.when(pl.col('totalPrice') <= pl.col('price_q25'))
            .then(1)  # Budget tier
            .when(pl.col('totalPrice') <= pl.col('price_q50'))
            .then(2)  # Economy tier
            .when(pl.col('totalPrice') <= pl.col('price_q75'))
            .then(3)  # Premium tier
            .otherwise(4)  # Luxury tier
            .std()
            .fill_null(0)  # Handle null case
            / 3
        ).clip(0, 1)).alias('consistent_price_tier'),

        # Most common price tier
        pl.when(pl.col('totalPrice') <= pl.col('price_q25'))
        .then(1)  # Budget tier
        .when(pl.col('totalPrice') <= pl.col('price_q50'))
        .then(2)  # Economy tier
        .when(pl.col('totalPrice') <= pl.col('price_q75'))
        .then(3)  # Premium tier
        .otherwise(4)  # Luxury tier
        .mode().first().alias('preferred_price_tier'),
    ]


def create_service_preference_features() -> List[pl.Expr]:
    """Create features related to service preferences."""
    # First get all the relevant column names for type and quantity
    type_cols = [
        f'legs{leg}_segments{seg}_baggageAllowance_weightMeasurementType'
        for leg in range(2) for seg in range(4)
    ]
    qty_cols = [
        f'legs{leg}_segments{seg}_baggageAllowance_quantity'
        for leg in range(2) for seg in range(4)
    ]

    return [
        # Baggage quantity preference (average of minimum bags allowed per flight option)
        pl.min_horizontal([
            pl.when(pl.col(type_col) == 0)
            .then(pl.col(qty_col))
            .otherwise(pl.lit(None))
            for type_col, qty_col in zip(type_cols, qty_cols)
        ]).mean().fill_null(0).alias('baggage_qty_preference'),

        # Baggage weight preference (average of minimum weight allowed per flight option)
        pl.min_horizontal([
            pl.when(pl.col(type_col) == 1)
            .then(pl.col(qty_col))
            .otherwise(pl.lit(None))
            for type_col, qty_col in zip(type_cols, qty_cols)
        ]).mean().fill_null(0).fill_nan(0).alias('baggage_weight_preference'),

        # Loyalty program utilization
        pl.col('frequentFlyer').is_not_null().mean().alias('loyalty_program_utilization')
    ]


def create_derived_metrics() -> List[pl.Expr]:
    """Create complex derived metrics from combinations of features."""
    return [
        # Convenience priority score (higher = more emphasis on convenient times)
        ((1 - pl.col('time_of_day_variance')) * 10 +
         pl.col('price_to_duration_sensitivity') * 5)
        .alias('convenience_priority_score'),

        # Loyalty vs price index (higher = more loyal, less price sensitive)
        (pl.col('loyalty_program_utilization') * 10 -
         pl.col('price_position_preference') / 10)
        .alias('loyalty_vs_price_index'),

        # Planning consistency score (inverse of lead time variance)
        (1 / (pl.col('max_booking_lead_days') - pl.col('min_booking_lead_days') + 1))
        .alias('planning_consistency_score'),

        # Luxury index (combination of cabin class and price tier)
        (pl.col('avg_cabin_class') * 20 +
         pl.col('price_position_preference') / 2)
        .alias('luxury_index'),

        # Create advanced features
        (pl.col('total_searches') / pl.col('unique_routes_searched').clip(1)).alias('search_intensity_per_route'),
        (pl.col('max_booking_lead_days') - pl.col('min_booking_lead_days')).alias('lead_time_variance'),
        (pl.col('avg_booking_lead_days') / pl.col('median_booking_lead_days').clip(1)).alias('lead_time_skew'),
        (pl.col('unique_carriers_used') / pl.col('total_searches').clip(1)).alias('carrier_diversity'),
        (pl.col('unique_departure_airports') / pl.col('total_searches').clip(1)).alias('airport_diversity'),
        (pl.col('max_cabin_class') - pl.col('min_cabin_class')).alias('cabin_class_range'),
        (pl.col('is_vip').cast(pl.Int8) * 2 + pl.col('has_corp_codes').is_not_null().cast(pl.Int8)).alias('customer_tier'),
    ]


def add_trip_duration(df: pl.DataFrame) -> pl.DataFrame:
    """Add trip duration to the DataFrame."""
    leg0_duration_minutes = parse_duration_to_minutes('legs0_duration')
    leg1_duration_minutes = parse_duration_to_minutes('legs1_duration')
    trip_duration_minutes = leg0_duration_minutes + leg1_duration_minutes

    return df.with_columns([
        leg0_duration_minutes.alias('leg0_duration_minutes'),
        leg1_duration_minutes.alias('leg1_duration_minutes'),
        trip_duration_minutes.alias('trip_duration_minutes'),
    ])


def create_windows_based_features(df) -> pl.DataFrame:
    # Add window-based features like price_percentile if they don't exist
    return (df.with_columns([
        # calculate price percentile over search session
        ((pl.col('totalPrice').rank(method='min').over('ranker_id') - 1) /
        (pl.col('totalPrice').count().over('ranker_id') - 1) * 100)
        .fill_null(50.0).alias('price_percentile'),

        # calculate price quartiles over profileId
        pl.col('totalPrice').quantile(0.25).over('profileId').alias('price_q25'),
        pl.col('totalPrice').quantile(0.50).over('profileId').alias('price_q50'),
        pl.col('totalPrice').quantile(0.75).over('profileId').alias('price_q75'),

        # calculate leg0_duration quartiles over profileId
        pl.col('leg0_duration_minutes').quantile(0.25).over('profileId').alias('leg0_duration_q25'),
        pl.col('leg0_duration_minutes').quantile(0.50).over('profileId').alias('leg0_duration_q50'),
        pl.col('leg0_duration_minutes').quantile(0.75).over('profileId').alias('leg0_duration_q75'),
    ]))


def create_interaction_features() -> List[pl.Expr]:
    # Create customer/business interaction features
    return [
        # Create VIP interactions
        (pl.col('search_intensity_per_route') * pl.col('is_vip')).alias('vip_search_intensity'),
        (pl.col('carrier_diversity') * pl.col('is_vip')).alias('vip_carrier_diversity'),
        (pl.col('avg_cabin_class') * pl.col('is_vip')).alias('vip_cabin_preference'),

        # Create corporate interactions
        (pl.col('total_searches') * pl.col('has_corp_codes')).alias('corp_search_volume'),
        (pl.col('roundtrip_preference') * pl.col('has_corp_codes')).alias('corp_roundtrip_pref'),
        (pl.col('lead_time_variance') * pl.col('has_corp_codes')).alias('corp_planning_variance'),
    ]

def extract_customer_features(df: pl.DataFrame) -> pl.DataFrame:
    """
    Extract customer features for clustering analysis.
    Aggregates by profileId to create customer-level features.
    """
    # Check if already processed
    if df.height > 0 and 'total_searches' in df.columns:
        return df

    # Get cabin class columns
    cabin_class_cols = [col for col in df.columns if col.startswith('legs') and col.endswith('_cabinClass')]

    # Create lazy frame and group by profileId
    lazy_df = create_windows_based_features(add_trip_duration(df)).lazy().group_by('profileId')

    # Apply customer feature groups
    lazy_df = lazy_df.agg([
        *create_customer_aggregation_features(),
        *create_booking_lead_time_features(),
        *create_travel_preference_features(),
        *create_cabin_class_features(cabin_class_cols),
        *create_temporal_preference_features(),
        *create_route_specific_features(),
        *create_price_sensitivity_features(),
        *create_service_preference_features()
    ])

    # Materialize to generate the basic features
    # base_features = customer_features.collect()

    # Add the derived metrics that depend on the generated features
    lazy_df = lazy_df.with_columns(create_derived_metrics())

    # Add interactive features
    lazy_df = lazy_df.with_columns(create_interaction_features())

    # print(f"Generated {len(enhanced_features.columns)} customer features for {len(enhanced_features)} customers")
    return lazy_df.collect().fill_null(0).fill_nan(0)

In [7]:
cust_data = extract_customer_features(data)
print(f'Generated {len(cust_data.columns)} customer features for {len(cust_data)} customers')

Generated 58 customer features for 32922 customers


In [8]:
cust_data.head(100)

profileId,company_id,sex,nationality,frequent_flyer,is_vip,by_self,has_corp_codes,total_searches,roundtrip_preference,unique_routes_searched,min_booking_lead_days,max_booking_lead_days,avg_booking_lead_days,median_booking_lead_days,most_common_departure_airport,unique_departure_airports,most_common_carrier,unique_carriers_used,min_cabin_class,max_cabin_class,avg_cabin_class,weekday_preference,weekend_travel_rate,time_of_day_variance,night_flight_preference,route_loyalty,hub_preference,short_haul_preference,connection_tolerance,preferred_duration_quartile,price_to_duration_sensitivity,avg_price_per_minute,price_per_minute_variance,price_position_preference,premium_economy_preference,consistent_price_tier,preferred_price_tier,baggage_qty_preference,baggage_weight_preference,loyalty_program_utilization,convenience_priority_score,loyalty_vs_price_index,planning_consistency_score,luxury_index,search_intensity_per_route,lead_time_variance,lead_time_skew,carrier_diversity,airport_diversity,cabin_class_range,customer_tier,vip_search_intensity,vip_carrier_diversity,vip_cabin_preference,corp_search_volume,corp_roundtrip_pref,corp_planning_variance
i64,i64,i8,i64,str,i8,i8,i8,u32,f64,u32,i32,i32,f64,f64,str,u32,str,u32,f64,f64,f64,i8,f64,f64,f64,f64,f64,f64,f64,i32,f64,f64,f64,f64,f64,f64,i32,f64,f64,f64,f64,f64,f64,f64,f64,i32,f64,f64,f64,f64,i8,f64,f64,f64,i64,f64,i32
3547026,62262,1,36,"""SU""",0,1,0,464,1.0,1,30,31,30.549569,31.0,"""DME""",4,"""U6""",5,1.0,2.0,1.072917,4,0.0,6.617567,0.344828,0.997845,0.0,0.0,1.700431,1,0.154816,49.496154,61.528717,49.050421,0.067888,0.634882,3,0.918103,0.0,1.0,-55.401591,5.094958,0.5,45.983544,464.0,1,0.98547,0.010776,0.008621,1.0,1,0.0,0.0,0.0,0,0.0,0
2468117,42620,1,36,"""""",0,1,1,517,1.0,1,1,2,1.448743,1.0,"""KUF""",1,"""SU""",7,1.0,1.0,1.0,3,0.0,5.138309,0.226306,0.998066,0.0,0.366226,1.05029,1,0.067044,114.55567,70.852326,49.775741,0.0,0.627839,1,0.479691,0.0,0.0,-41.047868,-4.977574,0.5,44.887871,517.0,1,1.448743,0.01354,0.001934,0.0,1,0.0,0.0,0.0,517,1.0,1
3294975,63398,1,36,"""""",0,1,1,12,1.0,1,16,17,16.666667,17.0,"""PEZ""",1,"""SU""",1,1.0,1.0,1.0,5,1.0,4.748205,0.333333,0.916667,0.0,0.65,1.0,1,0.420958,121.35422,28.21127,50.0,0.0,0.589695,1,1.5,0.0,0.0,-35.377263,-5.0,0.5,45.0,12.0,1,0.980392,0.083333,0.083333,0.0,1,0.0,0.0,0.0,12,1.0,1
2547292,58350,1,36,"""""",0,1,0,318,1.0,2,8,13,9.427673,8.0,"""OVB""",1,"""S7""",7,1.0,1.0,1.0,3,0.0,5.615115,0.075472,0.993711,0.0,0.0,1.0,2,0.105628,121.580331,45.047243,47.60402,0.0,0.625869,2,1.062893,0.0,0.0,-45.623005,-4.760402,0.166667,43.80201,159.0,5,1.178459,0.022013,0.003145,0.0,1,0.0,0.0,0.0,0,0.0,0
2153918,40253,0,36,"""""",0,1,1,11,0.0,1,14,14,14.0,14.0,"""OVB""",1,"""S7""",1,1.0,1.0,1.0,1,0.0,3.744086,0.0,0.909091,0.0,0.0,1.818182,1,0.662494,46.663264,18.331365,47.272727,0.0,0.651845,1,0.636364,0.0,0.0,-24.128393,-4.727273,1.0,43.636364,11.0,0,1.0,0.090909,0.090909,0.0,1,0.0,0.0,0.0,11,0.0,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
1119689,27507,1,36,"""""",0,1,1,35,0.0,2,5,9,7.428571,8.0,"""NBC""",2,"""SU""",3,1.0,1.0,1.0,4,0.0,5.661012,0.257143,0.942857,0.0,0.249524,1.057143,3,0.730338,75.117679,34.884892,48.268398,0.0,0.618348,1,0.857143,0.0,0.0,-42.95843,-4.82684,0.2,44.134199,17.5,4,0.928571,0.085714,0.057143,0.0,1,0.0,0.0,0.0,35,0.0,4
3221567,59457,0,36,"""SU/UT/S7""",0,1,1,37,1.0,1,22,22,22.0,22.0,"""NJC""",1,"""7R""",4,1.0,1.0,1.0,3,0.0,2.088148,0.081081,0.972973,0.0,0.0,1.918919,1,0.006442,38.617445,20.45318,48.198198,0.0,0.611433,1,0.243243,0.0,1.0,-10.849265,5.18018,1.0,44.099099,37.0,0,1.0,0.108108,0.027027,0.0,1,0.0,0.0,0.0,37,1.0,0
2444394,53359,1,36,"""""",1,1,1,58,0.0,2,42,52,44.206897,45.0,"""IKT""",2,"""SU""",4,1.0,4.0,1.517241,6,1.0,4.156824,0.224138,0.965517,0.0,0.0,1.310345,1,0.720128,75.185378,46.901084,48.761271,0.0,0.615406,1,1.017241,20.0,0.0,-27.967602,-4.876127,0.090909,54.725463,29.0,10,0.982375,0.068966,0.034483,3.0,3,29.0,0.068966,1.517

In [26]:
data.filter(pl.col('profileId') == 1076167)

Id,bySelf,companyID,corporateTariffCode,frequentFlyer,nationality,isAccess3D,isVip,legs0_arrivalAt,legs0_departureAt,legs0_duration,legs0_segments0_aircraft_code,legs0_segments0_arrivalTo_airport_city_iata,legs0_segments0_arrivalTo_airport_iata,legs0_segments0_baggageAllowance_quantity,legs0_segments0_baggageAllowance_weightMeasurementType,legs0_segments0_cabinClass,legs0_segments0_departureFrom_airport_iata,legs0_segments0_duration,legs0_segments0_flightNumber,legs0_segments0_marketingCarrier_code,legs0_segments0_operatingCarrier_code,legs0_segments0_seatsAvailable,legs0_segments1_aircraft_code,legs0_segments1_arrivalTo_airport_city_iata,legs0_segments1_arrivalTo_airport_iata,legs0_segments1_baggageAllowance_quantity,legs0_segments1_baggageAllowance_weightMeasurementType,legs0_segments1_cabinClass,legs0_segments1_departureFrom_airport_iata,legs0_segments1_duration,legs0_segments1_flightNumber,legs0_segments1_marketingCarrier_code,legs0_segments1_operatingCarrier_code,legs0_segments1_seatsAvailable,legs0_segments2_aircraft_code,legs0_segments2_arrivalTo_airport_city_iata,…,legs1_segments2_baggageAllowance_weightMeasurementType,legs1_segments2_cabinClass,legs1_segments2_departureFrom_airport_iata,legs1_segments2_duration,legs1_segments2_flightNumber,legs1_segments2_marketingCarrier_code,legs1_segments2_operatingCarrier_code,legs1_segments2_seatsAvailable,legs1_segments3_aircraft_code,legs1_segments3_arrivalTo_airport_city_iata,legs1_segments3_arrivalTo_airport_iata,legs1_segments3_baggageAllowance_quantity,legs1_segments3_baggageAllowance_weightMeasurementType,legs1_segments3_cabinClass,legs1_segments3_departureFrom_airport_iata,legs1_segments3_duration,legs1_segments3_flightNumber,legs1_segments3_marketingCarrier_code,legs1_segments3_operatingCarrier_code,legs1_segments3_seatsAvailable,miniRules0_monetaryAmount,miniRules0_percentage,miniRules0_statusInfos,miniRules1_monetaryAmount,miniRules1_percentage,miniRules1_statusInfos,pricingInfo_isAccessTP,pricingInfo_passengerCount,profileId,ranker_id,requestDate,searchRoute,sex,taxes,totalPrice,selected,__index_level_0__
i64,bool,i64,i64,str,i64,bool,bool,str,str,str,str,str,str,f64,f64,f64,str,str,str,str,str,f64,str,str,str,f64,f64,f64,str,str,str,str,str,f64,str,str,…,f64,f64,str,str,str,str,str,f64,str,str,str,f64,f64,f64,str,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,str,datetime[ns],str,bool,f64,f64,i64,i64
1456197,true,43648,null,"""SU""",36,false,false,"""2024-06-26T15:40:00""","""2024-06-26T14:00:00""","""02:40:00""","""73H""","""KGD""","""KGD""",0.0,0.0,1.0,"""SVO""","""02:40:00""","""6835""","""DP""","""DP""",1.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1.0,1,1076167,"""3080ece7142443cc86655ac22dd609…",2024-06-07 15:27:54,"""MOWKGD/KGDMOW""",false,2352.0,16471.0,0,1456197
1456198,true,43648,null,"""SU""",36,false,false,"""2024-06-26T15:30:00""","""2024-06-26T13:55:00""","""02:35:00""","""738""","""KGD""","""KGD""",0.0,0.0,1.0,"""SVO""","""02:35:00""","""253""","""5N""","""5N""",9.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1.0,1,1076167,"""3080ece7142443cc86655ac22dd609…",2024-06-07 15:27:54,"""MOWKGD/KGDMOW""",false,605.0,17494.0,0,1456198
1456199,true,43648,null,"""SU""",36,false,false,"""2024-06-26T15:40:00""","""2024-06-26T14:00:00""","""02:40:00""","""73H""","""KGD""","""KGD""",0.0,0.0,1.0,"""SVO""","""02:40:00""","""6835""","""DP""","""DP""",1.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1.0,1,1076167,"""3080ece7142443cc86655ac22dd609…",2024-06-07 15:27:54,"""MOWKGD/KGDMOW""",false,5

In [64]:
from scipy.stats import zscore
from sklearn.cluster import AgglomerativeClustering
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.ensemble import IsolationForest
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.preprocessing import StandardScaler, RobustScaler
import umap


def remove_outliers(df, method='isolation_forest', contamination=0.05):
    """Remove outliers using various methods"""

    # Get numeric columns only
    numeric_cols = [col for col, dtype in df.schema.items() if dtype.is_numeric()]
    df_numeric = df.select(numeric_cols)

    if method == 'isolation_forest':
        iso_forest = IsolationForest(contamination=contamination, random_state=42)
        outlier_labels = iso_forest.fit_predict(df_numeric.to_pandas())
        mask = outlier_labels == 1

    elif method == 'zscore':
        # Remove rows where any feature has |z-score| > 3
        df_pd = df_numeric.to_pandas()
        z_scores = np.abs(zscore(df_pd, nan_policy='omit'))
        mask = (z_scores < 3).all(axis=1)

    elif method == 'iqr':
        # Remove rows outside 1.5*IQR for any feature
        df_pd = df_numeric.to_pandas()
        Q1 = df_pd.quantile(0.25)
        Q3 = df_pd.quantile(0.75)
        IQR = Q3 - Q1
        mask = ~((df_pd < (Q1 - 1.5 * IQR)) | (df_pd > (Q3 + 1.5 * IQR))).any(axis=1)

    # Get outlier indices
    outlier_indices = np.where(~mask)[0]

    cleaned_df = df.filter(pl.Series('mask', mask))
    print(f"Removed {len(df) - len(cleaned_df):,} outliers ({(len(df) - len(cleaned_df))/len(df)*100:.1f}%)")

    return cleaned_df, outlier_indices

def encode_features(df):
    # Get categorical columns
    categorical_cols = [col for col, dtype in df.schema.items() if not dtype.is_numeric() and col not in ['id', 'ranker_id', 'request_date']]

    encoded_df = df.clone()
    encoders = {}

    for col in categorical_cols:
        if col == 'frequent_flyer':
            # Special handling for frequent flyer programs (list feature)
            # Create binary features for most common programs
            all_programs = []
            for programs in df[col].to_list():
                if isinstance(programs, list):
                    all_programs.extend(programs)

            from collections import Counter
            top_programs = Counter(all_programs).most_common(10)

            for program, _ in top_programs:
                encoded_df = encoded_df.with_columns([
                    pl.col(col).map_elements(
                        lambda x: 1 if isinstance(x, list) and program in x else 0,
                        return_dtype=pl.Int8
                    ).alias(f'ff_{program}')
                ])

            # Add count of total programs
            encoded_df = encoded_df.with_columns([
                pl.col(col).map_elements(
                    lambda x: len(x) if isinstance(x, list) else 0,
                    return_dtype=pl.Int8
                ).alias('ff_program_count')
            ])

        else:
            # Target encoding for high-cardinality categoricals like airports/carriers
            if col in ['most_common_departure_airport', 'most_common_carrier']:
                # Use frequency encoding
                value_counts = df[col].value_counts()
                freq_map = {row[0]: row[1] for row in value_counts.rows()}

                encoded_df = encoded_df.with_columns([
                    pl.col(col).replace_strict(freq_map, default=0).alias(f'{col}_frequency')
                ])
            else:
                # Standard label encoding for low-cardinality features
                unique_values = df[col].fill_null('MISSING').unique().sort().to_list()
                encoders[col] = {val: idx for idx, val in enumerate(unique_values)}

                encoded_df = encoded_df.with_columns([
                    pl.col(col).fill_null('MISSING').replace(encoders[col]).alias(f'{col}_encoded')

                ])

    # Remove original categorical columns
    final_df = encoded_df.select(pl.exclude(categorical_cols + ['frequent_flyer', 'most_common_departure_airport', 'most_common_carrier']))

    return final_df.fill_null(0), encoders

def dimensionality_reduction(scaled_features, method='pca', n_components=50):
    """Apply dimensionality reduction before clustering"""

    if method == 'pca':
        reducer = PCA(n_components=n_components, random_state=42)

    elif method == 'truncated_svd':
        reducer = TruncatedSVD(n_components=n_components, random_state=42)

    elif method == 'umap':
        reducer = umap.UMAP(n_components=n_components, random_state=42, n_neighbors=15)

    reduced_features = reducer.fit_transform(scaled_features)

    if hasattr(reducer, 'explained_variance_ratio_'):
        total_variance = reducer.explained_variance_ratio_.sum()
        print(f"{method.upper()} retained {total_variance:.3f} of total variance with {n_components} components")

    return reduced_features, reducer


def generate_clusters(features, n_clusters=10):
    agg = AgglomerativeClustering(n_clusters=n_clusters, linkage='ward')
    agg_segments = agg.fit_predict(features)

    # Create pseudo-centroids for use in prediction, since AgglomerativeClustering doesn't have centroids
    unique_segments = np.unique(agg_segments)
    centroids = []
    for segment_id in unique_segments:
        segment_mask = agg_segments == segment_id
        centroid = features[segment_mask].mean(axis=0)
        centroids.append(centroid)

    centroids = np.array(centroids)

    score = silhouette_score(features, agg_segments)

    return {
        "labels": agg_segments,
        "centroids": centroids,
        "model": agg,
        "silhouette": score,
        "n_clusters": n_clusters
    }

### Generate Customer Clusters

In [63]:
def analyze_clusters(original_df, cluster_labels, outlier_indices=None):
    """Analyze the final clusters"""
    # Add cluster labels to original dataframe for analysis
    if original_df is not None:

        # Check if sizes match
        if len(cluster_labels) != (len(original_df) - (len(outlier_indices) if outlier_indices is not None else 0)):
            print(f"Warning: Cluster labels size ({len(cluster_labels)}) doesn't match DataFrame size ({len(original_df)})")
            print("This likely happened because outliers were removed during clustering and outlier indices not provided or incorrect.")

        else:
            if outlier_indices is not None and len(outlier_indices) > 0:
                mask = ~pl.Series(range(len(original_df))).is_in(outlier_indices)
                analysis_df = original_df.filter(mask)
            else:
                analysis_df = original_df.head(len(cluster_labels))

            # Now add cluster labels safely
            analysis_df = analysis_df.with_columns(pl.Series('cluster', cluster_labels))

            # Cluster profiling
            print("\n📊 CLUSTER PROFILES:")
            print("=" * 50)

            cluster_profiles = analysis_df.group_by('cluster').agg([
                pl.col('total_searches').mean().alias('avg_searches'),
                pl.col('is_vip').mean().alias('vip_rate'),
                pl.col('roundtrip_preference').mean().alias('roundtrip_rate'),
                pl.col('avg_booking_lead_days').mean().alias('avg_lead_days'),
                pl.col('unique_carriers_used').mean().alias('avg_carriers'),
                pl.len().alias('size')
            ]).sort('cluster')

            print(cluster_profiles)



In [33]:
# 1. Feature Engineering
cust_data = extract_customer_features(data)

In [12]:
cust_data.head(100)

profileId,company_id,sex,nationality,frequent_flyer,is_vip,by_self,has_corp_codes,total_searches,roundtrip_preference,unique_routes_searched,min_booking_lead_days,max_booking_lead_days,avg_booking_lead_days,median_booking_lead_days,most_common_departure_airport,unique_departure_airports,most_common_carrier,unique_carriers_used,min_cabin_class,max_cabin_class,avg_cabin_class,weekday_preference,weekend_travel_rate,time_of_day_variance,night_flight_preference,route_loyalty,hub_preference,short_haul_preference,connection_tolerance,preferred_duration_quartile,price_to_duration_sensitivity,avg_price_per_minute,price_per_minute_variance,price_position_preference,premium_economy_preference,consistent_price_tier,preferred_price_tier,baggage_qty_preference,baggage_weight_preference,loyalty_program_utilization,convenience_priority_score,loyalty_vs_price_index,planning_consistency_score,luxury_index,search_intensity_per_route,lead_time_variance,lead_time_skew,carrier_diversity,airport_diversity,cabin_class_range,customer_tier,vip_search_intensity,vip_carrier_diversity,vip_cabin_preference,corp_search_volume,corp_roundtrip_pref,corp_planning_variance
i64,i64,i8,i64,str,i8,i8,i8,u32,f64,u32,i32,i32,f64,f64,str,u32,str,u32,f64,f64,f64,i8,f64,f64,f64,f64,f64,f64,f64,i32,f64,f64,f64,f64,f64,f64,i32,f64,f64,f64,f64,f64,f64,f64,f64,i32,f64,f64,f64,f64,i8,f64,f64,f64,i64,f64,i32
3547026,62262,1,36,"""SU""",0,1,0,464,1.0,1,30,31,30.549569,31.0,"""DME""",4,"""U6""",5,1.0,2.0,1.072917,4,0.0,6.617567,0.344828,0.997845,0.0,0.0,1.700431,1,0.154816,49.496154,61.528717,49.050421,0.067888,0.634882,3,0.918103,0.0,1.0,-55.401591,5.094958,0.5,45.983544,464.0,1,0.98547,0.010776,0.008621,1.0,1,0.0,0.0,0.0,0,0.0,0
2468117,42620,1,36,"""""",0,1,1,517,1.0,1,1,2,1.448743,1.0,"""KUF""",1,"""SU""",7,1.0,1.0,1.0,3,0.0,5.138309,0.226306,0.998066,0.0,0.366226,1.05029,1,0.067044,114.55567,70.852326,49.775741,0.0,0.627839,1,0.479691,0.0,0.0,-41.047868,-4.977574,0.5,44.887871,517.0,1,1.448743,0.01354,0.001934,0.0,1,0.0,0.0,0.0,517,1.0,1
3294975,63398,1,36,"""""",0,1,1,12,1.0,1,16,17,16.666667,17.0,"""PEZ""",1,"""SU""",1,1.0,1.0,1.0,5,1.0,4.748205,0.333333,0.916667,0.0,0.65,1.0,1,0.420958,121.35422,28.21127,50.0,0.0,0.589695,1,1.5,0.0,0.0,-35.377263,-5.0,0.5,45.0,12.0,1,0.980392,0.083333,0.083333,0.0,1,0.0,0.0,0.0,12,1.0,1
2547292,58350,1,36,"""""",0,1,0,318,1.0,2,8,13,9.427673,8.0,"""OVB""",1,"""S7""",7,1.0,1.0,1.0,3,0.0,5.615115,0.075472,0.993711,0.0,0.0,1.0,2,0.105628,121.580331,45.047243,47.60402,0.0,0.625869,2,1.062893,0.0,0.0,-45.623005,-4.760402,0.166667,43.80201,159.0,5,1.178459,0.022013,0.003145,0.0,1,0.0,0.0,0.0,0,0.0,0
2153918,40253,0,36,"""""",0,1,1,11,0.0,1,14,14,14.0,14.0,"""OVB""",1,"""S7""",1,1.0,1.0,1.0,1,0.0,3.744086,0.0,0.909091,0.0,0.0,1.818182,1,0.662494,46.663264,18.331365,47.272727,0.0,0.651845,1,0.636364,0.0,0.0,-24.128393,-4.727273,1.0,43.636364,11.0,0,1.0,0.090909,0.090909,0.0,1,0.0,0.0,0.0,11,0.0,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
1119689,27507,1,36,"""""",0,1,1,35,0.0,2,5,9,7.428571,8.0,"""NBC""",2,"""SU""",3,1.0,1.0,1.0,4,0.0,5.661012,0.257143,0.942857,0.0,0.249524,1.057143,3,0.730338,75.117679,34.884892,48.268398,0.0,0.618348,1,0.857143,0.0,0.0,-42.95843,-4.82684,0.2,44.134199,17.5,4,0.928571,0.085714,0.057143,0.0,1,0.0,0.0,0.0,35,0.0,4
3221567,59457,0,36,"""SU/UT/S7""",0,1,1,37,1.0,1,22,22,22.0,22.0,"""NJC""",1,"""7R""",4,1.0,1.0,1.0,3,0.0,2.088148,0.081081,0.972973,0.0,0.0,1.918919,1,0.006442,38.617445,20.45318,48.198198,0.0,0.611433,1,0.243243,0.0,1.0,-10.849265,5.18018,1.0,44.099099,37.0,0,1.0,0.108108,0.027027,0.0,1,0.0,0.0,0.0,37,1.0,0
2444394,53359,1,36,"""""",1,1,1,58,0.0,2,42,52,44.206897,45.0,"""IKT""",2,"""SU""",4,1.0,4.0,1.517241,6,1.0,4.156824,0.224138,0.965517,0.0,0.0,1.310345,1,0.720128,75.185378,46.901084,48.761271,0.0,0.615406,1,1.017241,20.0,0.0,-27.967602,-4.876127,0.090909,54.725463,29.0,10,0.982375,0.068966,0.034483,3.0,3,29.0,0.068966,1.517

In [35]:
data.filter(pl.col('profileId') == 3498928)

Id,bySelf,companyID,corporateTariffCode,frequentFlyer,nationality,isAccess3D,isVip,legs0_arrivalAt,legs0_departureAt,legs0_duration,legs0_segments0_aircraft_code,legs0_segments0_arrivalTo_airport_city_iata,legs0_segments0_arrivalTo_airport_iata,legs0_segments0_baggageAllowance_quantity,legs0_segments0_baggageAllowance_weightMeasurementType,legs0_segments0_cabinClass,legs0_segments0_departureFrom_airport_iata,legs0_segments0_duration,legs0_segments0_flightNumber,legs0_segments0_marketingCarrier_code,legs0_segments0_operatingCarrier_code,legs0_segments0_seatsAvailable,legs0_segments1_aircraft_code,legs0_segments1_arrivalTo_airport_city_iata,legs0_segments1_arrivalTo_airport_iata,legs0_segments1_baggageAllowance_quantity,legs0_segments1_baggageAllowance_weightMeasurementType,legs0_segments1_cabinClass,legs0_segments1_departureFrom_airport_iata,legs0_segments1_duration,legs0_segments1_flightNumber,legs0_segments1_marketingCarrier_code,legs0_segments1_operatingCarrier_code,legs0_segments1_seatsAvailable,legs0_segments2_aircraft_code,legs0_segments2_arrivalTo_airport_city_iata,…,legs1_segments2_baggageAllowance_weightMeasurementType,legs1_segments2_cabinClass,legs1_segments2_departureFrom_airport_iata,legs1_segments2_duration,legs1_segments2_flightNumber,legs1_segments2_marketingCarrier_code,legs1_segments2_operatingCarrier_code,legs1_segments2_seatsAvailable,legs1_segments3_aircraft_code,legs1_segments3_arrivalTo_airport_city_iata,legs1_segments3_arrivalTo_airport_iata,legs1_segments3_baggageAllowance_quantity,legs1_segments3_baggageAllowance_weightMeasurementType,legs1_segments3_cabinClass,legs1_segments3_departureFrom_airport_iata,legs1_segments3_duration,legs1_segments3_flightNumber,legs1_segments3_marketingCarrier_code,legs1_segments3_operatingCarrier_code,legs1_segments3_seatsAvailable,miniRules0_monetaryAmount,miniRules0_percentage,miniRules0_statusInfos,miniRules1_monetaryAmount,miniRules1_percentage,miniRules1_statusInfos,pricingInfo_isAccessTP,pricingInfo_passengerCount,profileId,ranker_id,requestDate,searchRoute,sex,taxes,totalPrice,selected,__index_level_0__
i64,bool,i64,i64,str,i64,bool,bool,str,str,str,str,str,str,f64,f64,f64,str,str,str,str,str,f64,str,str,str,f64,f64,f64,str,str,str,str,str,f64,str,str,…,f64,f64,str,str,str,str,str,f64,str,str,str,f64,f64,f64,str,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,str,datetime[ns],str,bool,f64,f64,i64,i64
11483731,true,36998,null,null,36,false,false,"""2024-09-24T05:40:00""","""2024-09-23T21:25:00""","""04:15:00""","""73H""","""NOZ""","""NOZ""",1.0,0.0,1.0,"""DME""","""04:15:00""","""2625""","""S7""","""S7""",9.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2300.0,null,1.0,3500.0,null,1.0,1.0,1,3498928,"""023b61d7412b4d788cf04dc9fd5592…",2024-09-12 09:00:20,"""MOWNOZ/NOZMOW""",true,988.0,18998.0,0,11483731
11483732,true,36998,null,null,36,false,false,"""2024-09-24T05:40:00""","""2024-09-23T21:25:00""","""04:15:00""","""73H""","""NOZ""","""NOZ""",1.0,0.0,1.0,"""DME""","""04:15:00""","""2625""","""S7""","""S7""",9.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.0,null,1.0,0.0,null,1.0,1.0,1,3498928,"""023b61d7412b4d788cf04dc9fd5592…",2024-09-12 09:00:20,"""MOWNOZ/NOZMOW""",true,988.0,51438.0,0,11483732
11483733,true,36998,null,null,36,false,false,"""2024-09-24T06:45:00""","""2024-09-23T22:20:00""","""04:25:00""","""32A""","""NOZ""","""NOZ""",1.0,0.0,1.0,"""SVO""","""04:25:00""","""1458""","""SU""","""SU""",8.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2800.0,null,1.0,2800.0,null,1.0,1.0,1,3498928,"""023b61d7412b4d788cf04dc9fd5592…",2024-09-12 09:00:20,"""MOWNOZ/NOZMOW""",true,1222.0,2847

In [14]:
cust_eval = add_trip_duration(data.filter(pl.col('profileId') == 3498928))
cust_eval

Id,bySelf,companyID,corporateTariffCode,frequentFlyer,nationality,isAccess3D,isVip,legs0_arrivalAt,legs0_departureAt,legs0_duration,legs0_segments0_aircraft_code,legs0_segments0_arrivalTo_airport_city_iata,legs0_segments0_arrivalTo_airport_iata,legs0_segments0_baggageAllowance_quantity,legs0_segments0_baggageAllowance_weightMeasurementType,legs0_segments0_cabinClass,legs0_segments0_departureFrom_airport_iata,legs0_segments0_duration,legs0_segments0_flightNumber,legs0_segments0_marketingCarrier_code,legs0_segments0_operatingCarrier_code,legs0_segments0_seatsAvailable,legs0_segments1_aircraft_code,legs0_segments1_arrivalTo_airport_city_iata,legs0_segments1_arrivalTo_airport_iata,legs0_segments1_baggageAllowance_quantity,legs0_segments1_baggageAllowance_weightMeasurementType,legs0_segments1_cabinClass,legs0_segments1_departureFrom_airport_iata,legs0_segments1_duration,legs0_segments1_flightNumber,legs0_segments1_marketingCarrier_code,legs0_segments1_operatingCarrier_code,legs0_segments1_seatsAvailable,legs0_segments2_aircraft_code,legs0_segments2_arrivalTo_airport_city_iata,…,legs1_segments2_duration,legs1_segments2_flightNumber,legs1_segments2_marketingCarrier_code,legs1_segments2_operatingCarrier_code,legs1_segments2_seatsAvailable,legs1_segments3_aircraft_code,legs1_segments3_arrivalTo_airport_city_iata,legs1_segments3_arrivalTo_airport_iata,legs1_segments3_baggageAllowance_quantity,legs1_segments3_baggageAllowance_weightMeasurementType,legs1_segments3_cabinClass,legs1_segments3_departureFrom_airport_iata,legs1_segments3_duration,legs1_segments3_flightNumber,legs1_segments3_marketingCarrier_code,legs1_segments3_operatingCarrier_code,legs1_segments3_seatsAvailable,miniRules0_monetaryAmount,miniRules0_percentage,miniRules0_statusInfos,miniRules1_monetaryAmount,miniRules1_percentage,miniRules1_statusInfos,pricingInfo_isAccessTP,pricingInfo_passengerCount,profileId,ranker_id,requestDate,searchRoute,sex,taxes,totalPrice,selected,__index_level_0__,leg0_duration_minutes,leg1_duration_minutes,trip_duration_minutes
i64,bool,i64,i64,str,i64,bool,bool,str,str,str,str,str,str,f64,f64,f64,str,str,str,str,str,f64,str,str,str,f64,f64,f64,str,str,str,str,str,f64,str,str,…,str,str,str,str,f64,str,str,str,f64,f64,f64,str,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,str,datetime[ns],str,bool,f64,f64,i64,i64,i32,i32,i32
11483731,true,36998,null,null,36,false,false,"""2024-09-24T05:40:00""","""2024-09-23T21:25:00""","""04:15:00""","""73H""","""NOZ""","""NOZ""",1.0,0.0,1.0,"""DME""","""04:15:00""","""2625""","""S7""","""S7""",9.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2300.0,null,1.0,3500.0,null,1.0,1.0,1,3498928,"""023b61d7412b4d788cf04dc9fd5592…",2024-09-12 09:00:20,"""MOWNOZ/NOZMOW""",true,988.0,18998.0,0,11483731,255,285,540
11483732,true,36998,null,null,36,false,false,"""2024-09-24T05:40:00""","""2024-09-23T21:25:00""","""04:15:00""","""73H""","""NOZ""","""NOZ""",1.0,0.0,1.0,"""DME""","""04:15:00""","""2625""","""S7""","""S7""",9.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.0,null,1.0,0.0,null,1.0,1.0,1,3498928,"""023b61d7412b4d788cf04dc9fd5592…",2024-09-12 09:00:20,"""MOWNOZ/NOZMOW""",true,988.0,51438.0,0,11483732,255,285,540
11483733,true,36998,null,null,36,false,false,"""2024-09-24T06:45:00""","""2024-09-23T22:20:00""","""04:25:00""","""32A""","""NOZ""","""NOZ""",1.0,0.0,1.0,"""SVO""","""04:25:00""","""1458""","""SU""","""SU""",8.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2800.0,null,1.0,2800.0,null,1.0,1.0,1,3498928,"""023b61d7412b4d788cf04dc9fd5592…",2024-09-12 09:00:20,"""MOWNOZ/NOZMOW""",true,1222.0,28472.0,1,11483733,265,275,540
11483734,true,36998,null,null,36,false,false,"""2024-

In [15]:
cust_eval.select(
    pl.corr(
        pl.col('totalPrice'),
        pl.col('trip_duration_minutes')
    ).fill_null(0)
)


totalPrice
f64
NaN


In [56]:
# 2. Remove outliers
cust_data_cleaned, outlier_indices = remove_outliers(cust_data, method='isolation_forest')

Removed 1,647 outliers (5.0%)


In [57]:
# 3. Advanced encoding
cust_data_encoded, encoders = encode_features(cust_data_cleaned)

In [49]:
# 2b/3b. Do not remove outliers
outlier_indices = []
cust_data_encoded, encoders = encode_features(cust_data)

In [58]:
# 4. Feature scaling with robust scaler
scaler = RobustScaler()  # Less sensitive to outliers than StandardScaler
scaled_features = scaler.fit_transform(cust_data_encoded.to_pandas())

In [59]:
scaled_features

array([[ 0.41393055,  0.48572388,  0.        , ...,  0.        ,
         0.04952027, -1.17330046],
       [-0.42016866, -0.61392901,  0.        , ...,  0.        ,
        -0.02847416,  0.        ],
       [ 0.21907117,  0.54932258,  0.        , ...,  0.        ,
        -0.25750542,  0.        ],
       ...,
       [ 0.4427987 ,  0.20904714,  0.        , ...,  0.        ,
        -0.2537914 ,  0.        ],
       [-0.58191793,  0.16487515, -1.        , ...,  0.        ,
         1.73909006,  0.        ],
       [-0.18234297,  0.03538238,  0.        , ...,  0.        ,
         1.73909006,  0.        ]], shape=(31275, 58))

In [52]:
print(scaled_features)
nan_mask = np.isnan(scaled_features)
print(np.where(nan_mask.any(axis=1)))

[[ 0.42854339  0.49577291  0.         ...  0.          0.01967695
  -1.16689121]
 [-0.39939875 -0.60394155  0.         ...  0.         -0.06402349
   0.        ]
 [ 0.2351224   0.55937517  0.         ...  0.         -0.2825257
   0.        ]
 ...
 [ 0.45719844  0.21908068  0.         ...  0.         -0.27988253
   0.        ]
 [-0.55995403  0.17490622 -1.         ...  0.          1.74214391
   0.        ]
 [-0.16332862  0.04540619  0.         ...  0.          1.74214391
   0.        ]]
(array([], dtype=int64),)


In [60]:
# 5. Dimensionality reduction
n_components = 30
reducer = PCA(n_components=n_components, random_state=42)
reduced_features = reducer.fit_transform(scaled_features)

if hasattr(reducer, 'explained_variance_ratio_'):
    total_variance = reducer.explained_variance_ratio_.sum()
    print(f"PCA retained {total_variance:.3f} of total variance with {n_components} components")

PCA retained 0.999 of total variance with 30 components


In [61]:
# 6. Apply clustering algorithm
clustering_results = generate_clusters(reduced_features)

# 7. Clustering results
print("\n🏆 CLUSTERING RESULTS:")
print(f"  Clusters: {clustering_results['n_clusters']:2d}| Silhouette: {clustering_results['silhouette']:.4f} | ")


🏆 CLUSTERING RESULTS:
  Clusters: 10| Silhouette: 0.5476 | 


In [65]:
analyze_clusters(cust_data, clustering_results['labels'], outlier_indices)


📊 CLUSTER PROFILES:
shape: (10, 7)
┌─────────┬──────────────┬──────────┬────────────────┬───────────────┬──────────────┬───────┐
│ cluster ┆ avg_searches ┆ vip_rate ┆ roundtrip_rate ┆ avg_lead_days ┆ avg_carriers ┆ size  │
│ ---     ┆ ---          ┆ ---      ┆ ---            ┆ ---           ┆ ---          ┆ ---   │
│ i64     ┆ f64          ┆ f64      ┆ f64            ┆ f64           ┆ f64          ┆ u32   │
╞═════════╪══════════════╪══════════╪════════════════╪═══════════════╪══════════════╪═══════╡
│ 0       ┆ 611.653556   ┆ 0.0      ┆ 0.51406        ┆ 15.942428     ┆ 9.076509     ┆ 3712  │
│ 1       ┆ 324.402778   ┆ 1.0      ┆ 0.670224       ┆ 11.339924     ┆ 5.402778     ┆ 72    │
│ 2       ┆ 454.712367   ┆ 0.002865 ┆ 0.572092       ┆ 14.694453     ┆ 4.668905     ┆ 22341 │
│ 3       ┆ 622.083333   ┆ 1.0      ┆ 0.681143       ┆ 11.606121     ┆ 6.041667     ┆ 24    │
│ 4       ┆ 1095.0       ┆ 1.0      ┆ 1.0            ┆ 15.543681     ┆ 6.333333     ┆ 3     │
│ 5       ┆ 867.444444  

### Engineer Flight Features

In [34]:
CUSTOMER_FEATURES = ['companyID', 'sex', 'nationality', 'isVip', 'bySelf']
UNNEEDED_FEATURES = [
    'frequentFlyer', 'frequent_flyer', 'isAccess3D', 'requestDate', 'searchRoute', 'ranker_id', 'totalPrice', 'taxes', 'legs0_arrivalAt', 'legs0_duration'
]
UNNEEDED_FEATURES_REGEX = r'^legs[01].*$|^leg0_duration_q.*$|^price_q.*$'
POLARS_INDEX_COL = ['__index_level_0__']


# function to get column groups for the flight segments features that will be used in engineering the new features
def get_column_groups(df: pl.DataFrame) -> Dict[str, List[str]]:
    columns = df.columns
    return {
        'leg0_duration': [col for col in columns if col.startswith('legs0_segments') and col.endswith('_duration')],
        'leg1_duration': [col for col in columns if col.startswith('legs1_segments') and col.endswith('_duration')],
        'mkt_carrier': [col for col in columns if col.endswith('marketingCarrier_code')],
        'op_carrier': [col for col in columns if col.endswith('operatingCarrier_code')],
        'aircraft': [col for col in columns if col.endswith('aircraft_code')],
        'airport': [col for col in columns if 'airport_iata' in col or 'airport_city_iata' in col],
        'seats_avail': [col for col in columns if col.endswith('seatsAvailable')],
        'baggage_type': [col for col in columns if 'baggageAllowance_weightMeasurementType' in col],
        'baggage_allowance': [col for col in columns if 'baggageAllowance_quantity' in col],
        'cabin_class': [col for col in columns if 'cabinClass' in col]
    }


def create_basic_features(col_groups: Dict[str, List[str]]) -> List[pl.Expr]:
    """Create basic customer and route characteristics features."""
    return [
        pl.col('isAccess3D').cast(pl.Int32).alias('is_access3D'),

        # Normalized frequentFlyer program, addressing null values as null strings, and translating UT program
        pl.col('frequentFlyer').str.replace('- ЮТэйр ЗАО', 'UT').fill_null('').alias('frequent_flyer'),

        # Route characteristics
        pl.col('legs1_departureAt').is_not_null().cast(pl.Int8).alias('is_roundtrip'),
        pl.col('searchRoute').str.slice(0, 3).alias('route_origin'),

        # Hub features
        pl.col('searchRoute').str.slice(0, 3).is_in(MAJOR_HUBS).cast(pl.Int32).alias('origin_is_major_hub'),
        pl.col('searchRoute').str.slice(3, 3).is_in(MAJOR_HUBS).cast(pl.Int32).alias('destination_is_major_hub'),
        pl.max_horizontal([
            pl.col(col).is_in(MAJOR_HUBS) for col in col_groups['airport']
        ]).cast(pl.Int32).alias('includes_major_hub'),
    ]


def create_segment_features(col_groups: Dict[str, List[str]]) -> List[pl.Expr]:
    """Create flight segment related features."""
    return [
        # Number of segments per leg
        pl.concat_list([pl.col(col) for col in col_groups['leg0_duration']])
        .list.drop_nulls().list.len().alias('leg0_num_segments'),

        pl.concat_list([pl.col(col) for col in col_groups['leg1_duration']])
        .list.drop_nulls().list.len().alias('leg1_num_segments'),

        # Total segments
        pl.sum_horizontal([pl.col(col).is_not_null() for col in col_groups['aircraft']])
        .alias('total_segments'),

        # Flight time in minutes (sum of segment durations)
        pl.sum_horizontal([
            parse_duration_to_minutes(col)
            for col in col_groups['leg0_duration']
        ]).alias('leg0_flight_time_min'),

        pl.sum_horizontal([
            parse_duration_to_minutes(col)
            for col in col_groups['leg1_duration']
        ]).alias('leg1_flight_time_min'),
    ]


def create_time_features() -> List[pl.Expr]:
    """Create time-based features."""
    return [
        # Booking lead time
        ((pl.col('legs0_departureAt').str.to_datetime() -
            pl.col('requestDate').cast(pl.Datetime)) / pl.duration(days=1)).cast(pl.Int32).alias('booking_lead_days'),

        # Trip duration features
        parse_duration_to_minutes('legs0_duration').alias('leg0_duration_minutes'),
        parse_duration_to_minutes('legs1_duration').alias('leg1_duration_minutes'),
        (parse_duration_to_minutes('legs0_duration') + parse_duration_to_minutes('legs1_duration')).alias('trip_duration_minutes'),

        # Departure/arrival hours
        pl.col('legs0_departureAt').str.to_datetime().dt.hour().alias('leg0_departure_hour'),
        pl.col('legs0_departureAt').str.to_datetime().dt.weekday().alias('leg0_departure_weekday'),
        pl.col('legs0_arrivalAt').str.to_datetime().dt.hour().alias('leg0_arrival_hour'),
        pl.col('legs0_arrivalAt').str.to_datetime().dt.weekday().alias('leg0_arrival_weekday'),
        pl.col('legs1_departureAt').str.to_datetime().dt.hour().alias('leg1_departure_hour'),
        pl.col('legs1_arrivalAt').str.to_datetime().dt.hour().alias('leg1_arrival_hour'),
        pl.col('legs1_departureAt').str.to_datetime().dt.weekday().alias('leg1_departure_weekday'),
        pl.col('legs1_arrivalAt').str.to_datetime().dt.weekday().alias('leg1_arrival_weekday'),
    ]


def create_carrier_features(col_groups: Dict[str, List[str]]) -> List[pl.Expr]:
    """Create carrier and aircraft features."""
    # Create combined carrier column group for carrier features
    all_carrier_cols = col_groups['mkt_carrier'] + col_groups['op_carrier']

    # Create frequent flyer matching expressions
    ff_matches_mkt = [
        pl.when(pl.col(col).is_not_null() & (pl.col(col) != ''))
        .then(pl.col('frequent_flyer').str.contains(pl.col(col)))
        .otherwise(False)
        for col in col_groups['mkt_carrier']
    ]
    ff_matches_op = [
        pl.when(pl.col(col).is_not_null() & (pl.col(col) != ''))
        .then(pl.col('frequent_flyer').str.contains(pl.col(col)))
        .otherwise(False)
        for col in col_groups['op_carrier']
    ]

    return [
        # Carrier features
        pl.concat_list([pl.col(col) for col in all_carrier_cols])
        .list.drop_nulls().list.unique().list.len().alias('unique_carriers'),

        pl.coalesce([pl.col(col) for col in all_carrier_cols]).alias('primary_carrier'),
        pl.col('legs0_segments0_marketingCarrier_code').alias('marketing_carrier'),

        # Frequent flyer matching (check if FF programs match carriers)
        pl.max_horizontal(ff_matches_mkt).cast(pl.Int32).alias('has_mkt_ff'),
        pl.max_horizontal(ff_matches_op).cast(pl.Int32).alias('has_op_ff'),

        # Aircraft features
        pl.concat_list([pl.col(col) for col in col_groups['aircraft']])
        .list.drop_nulls().list.unique().list.len().alias('aircraft_diversity'),

        pl.coalesce([pl.col(col) for col in col_groups['aircraft']]).alias('primary_aircraft'),
    ]


def create_service_features(col_groups: Dict[str, List[str]]) -> List[pl.Expr]:
    """Create service-related features (fees, cabin, seats, baggage)."""
    return [
        # Cancellation/exchange fees
        (
            ((pl.col('miniRules0_monetaryAmount') > 0) & pl.col('miniRules0_monetaryAmount').is_not_null()) |
            ((pl.col('miniRules0_percentage') > 0) & pl.col('miniRules0_percentage').is_not_null())
        ).cast(pl.Int32).alias('has_cancellation_fee'),

        (
            ((pl.col('miniRules1_monetaryAmount') > 0) & pl.col('miniRules1_monetaryAmount').is_not_null()) |
            ((pl.col('miniRules1_percentage') > 0) & pl.col('miniRules1_percentage').is_not_null())
        ).cast(pl.Int32).alias('has_exchange_fee'),

        # Cabin class features
        pl.max_horizontal([pl.col(col) for col in col_groups['cabin_class']]).alias('max_cabin_class'),
        pl.mean_horizontal([pl.col(col) for col in col_groups['cabin_class']]).alias('avg_cabin_class'),

        # Seat availability (using for understanding popularity/scarcity)
        pl.min_horizontal([pl.col(col) for col in col_groups['seats_avail']]).alias('min_seats_available'),
        pl.sum_horizontal([pl.col(col).fill_null(0) for col in col_groups['seats_avail']]).alias('total_seats_available'),

        # Baggage allowance
        pl.min_horizontal([
            pl.when(pl.col(type_col) == 0).then(pl.col(qty_col)).otherwise(pl.lit(None))
            for type_col, qty_col in zip(col_groups['baggage_type'], col_groups['baggage_allowance'])
        ]).fill_null(0).alias("baggage_allowance_quantity"),

        pl.min_horizontal([
            pl.when(pl.col(type_col) == 1).then(pl.col(qty_col)).otherwise(pl.lit(None))
            for type_col, qty_col in zip(col_groups['baggage_type'], col_groups['baggage_allowance'])
        ]).fill_null(0).alias("baggage_allowance_weight")
    ]


def add_route_popularity(lazy_df: pl.LazyFrame) -> pl.LazyFrame:
    """Add search route popularity features."""
    return lazy_df.with_columns([
        pl.len().over('searchRoute').alias('route_popularity')
    ]).with_columns([
        pl.col('route_popularity').log1p().alias('route_popularity_log')
    ])


def create_window_based_flight_features(lazy_df: pl.LazyFrame) -> pl.LazyFrame:
    return lazy_df.with_columns([
        # calculate price percentile over search session
        ((pl.col('totalPrice').rank(method='min').over('ranker_id') - 1) /
        (pl.col('totalPrice').count().over('ranker_id') - 1) * 100)
        .fill_null(50.0).alias('price_percentile'),

        # Price features (always relative to search session)
        (pl.col('totalPrice').rank(method='ordinal').over('ranker_id') /
         pl.col('totalPrice').count().over('ranker_id')).alias('price_rank_pct'),
        (pl.col('totalPrice') / pl.col('totalPrice').min().over('ranker_id')).alias('price_ratio_to_min'),
    ]).with_columns([

        # Route popularity features
        pl.len().over('searchRoute').alias('route_popularity')
    ]).with_columns([
        pl.col('route_popularity').log1p().alias('route_popularity_log')
    ])


def create_derived_flight_features() -> List[pl.Expr]:
    return [
        pl.col('leg0_departure_hour').is_between(6, 22).cast(pl.Int8).alias('is_daytime'),
        (pl.col('leg1_departure_weekday') >= 5).cast(pl.Int8).alias('is_weekend'),
        (pl.col('leg0_departure_hour').is_between(6, 22) &
         (~pl.col('leg0_arrival_hour').is_between(6, 22)))
        .cast(pl.Int8).alias('is_redeye'),
        (pl.col('total_segments') > 1).cast(pl.Int8).alias('has_connections'),
    ]

def extract_flight_features(df: pl.DataFrame) -> pl.DataFrame:
    """ Create flight-level features"""
    # Check if already processed
    if df.height > 0 and 'total_duration_mins' in df.columns:
        return df


    # Get column groups
    col_groups = get_column_groups(df)

    # Start with lazy frame, dropping unnecessary columns
    lazy_df = df.drop(CUSTOMER_FEATURES + POLARS_INDEX_COL, strict=False).lazy()
    lazy_df = create_window_based_flight_features(lazy_df)

    # Apply all feature groups
    lazy_df = lazy_df.with_columns([
        *create_basic_features(col_groups),
        *create_segment_features(col_groups),
        *create_time_features(),
        *create_service_features(col_groups)
    ])

    # Add carrier features (requires frequent_flyer from create_basic_features() step)
    lazy_df = lazy_df.with_columns(create_carrier_features(col_groups))

    # Add derived features
    lazy_df = lazy_df.with_columns(create_derived_flight_features())

    # Add window based derived flight features
    lazy_df = create_window_based_flight_features(lazy_df)

    # Materialize to generate new features, drop unused features, and fill nulls
    return (lazy_df
            .collect()
            .select(~cs.matches(UNNEEDED_FEATURES_REGEX) & ~cs.by_name(UNNEEDED_FEATURES))
            .fill_null(0)
            )


In [88]:
data = extract_flight_features(data)

### Prepare Training Data

Now combining customer features with clustering segmentation results and flight features

In [89]:
def prepare_training_data(cust_data, flight_features, cust_segments, outlier_indices=None):
    # if outliers were removed for clustering, then remove them for training
    if outlier_indices is not None and len(outlier_indices) > 0:
        mask = ~pl.Series(range(len(cust_data))).is_in(outlier_indices)
        cust_features = cust_data.filter(mask).lazy()
    else:
        cust_features = cust_data.lazy()

    # Add the cluster labels to the customer features
    cust_features = cust_features.with_columns([
        pl.Series('customer_segment', cust_segments)
    ])

    # Join the customer features with the flight features, removing outlier profiles
    flight_features = flight_features.lazy().join(cust_features, on='profileId', how='inner').drop(pl.col('profileId'))
    flight_features = flight_features.with_columns([
        (pl.col('is_daytime') * (1.0 - pl.col('night_flight_preference').fill_null(0.5))).alias('daytime_alignment'),
        (pl.col('is_weekend') * pl.col('weekend_travel_rate').fill_null(0.5)).alias('weekend_alignment'),
        (pl.col('price_rank_pct') * pl.col('price_position_preference').fill_null(0.5)).alias('price_preference_match'),
        (pl.col('primary_carrier') == pl.col('most_common_carrier').fill_null('unknown')).cast(pl.Int8).alias('carrier_loyalty_match'),
    ])

    # Materialize the data
    X = flight_features.collect()

    # Get target label
    y_true = X['selected'].to_numpy()

    return X.drop(pl.col('selected')), y_true


In [90]:
data, y_true = prepare_training_data(cust_data, data, clustering_results['labels'], outlier_indices)

In [91]:
data.head(1000)

Id,corporateTariffCode,miniRules0_monetaryAmount,miniRules0_percentage,miniRules0_statusInfos,miniRules1_monetaryAmount,miniRules1_percentage,miniRules1_statusInfos,pricingInfo_isAccessTP,pricingInfo_passengerCount,price_percentile,price_rank_pct,price_ratio_to_min,route_popularity,route_popularity_log,is_access3D,is_roundtrip,route_origin,origin_is_major_hub,destination_is_major_hub,includes_major_hub,leg0_num_segments,leg1_num_segments,total_segments,leg0_flight_time_min,leg1_flight_time_min,booking_lead_days,leg0_duration_minutes,leg1_duration_minutes,trip_duration_minutes,leg0_departure_hour,leg0_departure_weekday,leg0_arrival_hour,leg0_arrival_weekday,leg1_departure_hour,leg1_arrival_hour,leg1_departure_weekday,…,route_loyalty,hub_preference,short_haul_preference,connection_tolerance,preferred_duration_quartile,price_to_duration_sensitivity,avg_price_per_minute,price_per_minute_variance,price_position_preference,premium_economy_preference,consistent_price_tier,preferred_price_tier,baggage_qty_preference,baggage_weight_preference,loyalty_program_utilization,convenience_priority_score,loyalty_vs_price_index,planning_consistency_score,luxury_index,search_intensity_per_route,lead_time_variance,lead_time_skew,carrier_diversity,airport_diversity,cabin_class_range,customer_tier,vip_search_intensity,vip_carrier_diversity,vip_cabin_preference,corp_search_volume,corp_roundtrip_pref,corp_planning_variance,customer_segment,daytime_alignment,weekend_alignment,price_preference_match,carrier_loyalty_match
i64,i64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,u32,f64,i32,i8,str,i32,i32,i32,u32,u32,u32,i32,i32,i32,i32,i32,i32,i8,i8,i8,i8,i8,i8,i8,…,f64,f64,f64,f64,i32,f64,f64,f64,f64,f64,f64,i32,f64,f64,f64,f64,f64,f64,f64,f64,i32,f64,f64,f64,f64,i8,f64,f64,f64,i64,f64,i32,i64,f64,f64,f64,i8
0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,0.0,0.04,1.0,25,3.258097,0,1,"""TLK""",0,0,0,1,1,2,160,155,29,160,155,315,15,6,16,6,9,14,2,…,0.962264,0.0,0.0,1.981132,1,-0.253891,93.370333,68.147027,45.055998,0.141509,0.619049,1,1.018868,0.0,1.0,-32.58008,5.4944,0.052632,45.358188,26.5,18,1.433237,0.056604,0.028302,1.0,1,0.0,0.0,0.0,106,0.235849,18,6,1.0,0.0,1.80224,0
1,123,2300.0,0.0,1.0,3500.0,0.0,1.0,1.0,1,4.166667,0.08,3.028015,25,3.258097,1,1,"""TLK""",0,0,0,2,2,4,250,245,29,445,505,950,9,6,14,6,22,8,2,…,0.962264,0.0,0.0,1.981132,1,-0.253891,93.370333,68.147027,45.055998,0.141509,0.619049,1,1.018868,0.0,1.0,-32.58008,5.4944,0.052632,45.358188,26.5,18,1.433237,0.056604,0.028302,1.0,1,0.0,0.0,0.0,106,0.235849,18,6,1.0,0.0,3.60448,1
2,0,2300.0,0.0,1.0,3500.0,0.0,1.0,1.0,1,29.166667,0.32,3.18023,25,3.258097,0,1,"""TLK""",0,0,0,2,2,4,250,245,29,445,505,950,9,6,14,6,22,8,2,…,0.962264,0.0,0.0,1.981132,1,-0.253891,93.370333,68.147027,45.055998,0.141509,0.619049,1,1.018868,0.0,1.0,-32.58008,5.4944,0.052632,45.358188,26.5,18,1.433237,0.056604,0.028302,1.0,1,0.0,0.0,0.0,106,0.235849,18,6,1.0,0.0,14.417919,1
3,123,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1,54.166667,0.56,4.849562,25,3.258097,1,1,"""TLK""",0,0,0,2,2,4,250,245,29,445,505,950,9,6,14,6,22,8,2,…,0.962264,0.0,0.0,1.981132,1,-0.253891,93.370333,68.147027,45.055998,0.141509,0.619049,1,1.018868,0.0,1.0,-32.58008,5.4944,0.052632,45.358188,26.5,18,1.433237,0.056604,0.028302,1.0,1,0.0,0.0,0.0,106,0.235849,18,6,1.0,0.0,25.231359,1
4,0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1,79.166667,0.8,5.097726,25,3.258097,0,1,"""TLK""",0,0,0,2,2,4,250,245,29,445,505,950,9,6,14,6,22,8,2,…,0.962264,0.0,0.0,1.981132,1,-0.253891,93.370333,68.147027,45.055998,0.141509,0.619049,1,1.018868,0.0,1.0,-32.58008,5.4944,0.052632,45.358188,26.5,18,1.433237,0.056604,0.028302,1.0,1,0.0,0.0,0.0,106,0.235849,18,6,1.0,0.0,36.044799,1
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
997,0,2300.0,0.0,1.0,3500.0,0.0,1.0,0.0,1,72.47191,0.726257,3.73042,2913,7.977282,0,1,"""OVB""",0,0,0,2,2,4,630,655,3,690,870,1560,17,1,7,2,8,20,3,…,0.992629,0.0,0.0,1.307125,2,0.340584,89.040997,6

In [85]:
data.shape

(16040934, 120)

In [ ]:
# Create chunks for processing efficiency
def get_chunks(data, chunk_size=10000):
    X_chunks = []
    y_chunks = []
    group_chunks = []

    # Get unique ranker_ids to avoid splitting searches across groups
    ranker_ids = data.select('ranker_id').unique().sort('ranker_id')['ranker_id'].to_list()

    for i in range(0, len(ranker_ids), chunk_size):
        chunk_ranker_ids = ranker_ids[i:i + chunk_size]

        # Filter chunk by ranker_ids
        X_chunk = data.filter(pl.col('ranker_id').is_in(chunk_ranker_ids))

        y_chunk = X_chunk.select('selected').to_numpy().ravel()
        group_chunk = X_chunk.select('ranker_id').to_numpy().ravel()

        X_chunks.append(X_chunk)
        y_chunks.append(y_chunk)
        group_chunks.append(group_chunk)

    # Combine chunks
    print("Combining chunks...")
    X = np.vstack(X_chunks)
    y = np.concatenate(y_chunks)
    groups = np.concatenate(group_chunks)

    return np.concatenate(group_chunks)


In [ ]:
import lightgbm as lgb

# Create group sizes for LightGBM ranking
# unique_groups, group_sizes = np.unique(groups, return_counts=True)

# LightGBM with memory-efficient settings
lgb_model = lgb.LGBMRanker(
    objective='lambdarank',
    metric='ndcg',
    boosting_type='gbdt',
    num_leaves=31,
    learning_rate=0.1,
    feature_fraction=0.8,
    bagging_fraction=0.8,
    bagging_freq=5,
    verbose=-1,
    random_state=42,
    n_jobs=4,
    force_row_wise=True  # Memory efficient
)

lbg_model.fit(
    X, y,
    group=group_sizes,
    callbacks=[lgb.early_stopping(50), lgb.log_evaluation(100)]
)
